<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

http://httpbin.org/status/200,404,503,408,500,502,504

다양한 에러를 랜덤으로 테스트해볼 수 있음.

크롤링을 하다가 중간에 오류가 발생할 수 있기때문에, 크롤러 개발시 오류처리에 유의해야한다.

500번 에러가날 경우 답이 없다. 서버 개발자가 코드를 잘못 만들어서 나는 오류다.

In [6]:
import requests 
import time
from retrying import retry

In [7]:
# 일시적인 오류(네트워크 상태, 일시적인 장애)를 나타내는 상태코드(500번 제외)
# 500번은 서버 코드가 잘못돼 발생하는 코드
# 404 는 url 오류

error_code = (503,408,502,504,400)

In [8]:
# 요청처리를 하는 함수
# retry 는 힘수 실행도중 오류나 예외가 발생하면 해당 함수를 다시 호출하는 함수
# stop_max_attemp_number : 함수 최대 호출 횟수
# wait_expinential_multiplier : 함수 호출 간격 1000 ms = 1 초

@retry(stop_max_attempt_number=3, wait_exponential_multiplier=1000)

def fetch(url):
    
    # 요청
    res = requests.get(url)
    print(f'응답코드 : {res.status_code}')
    
    # 일시적 오류가 아니라면 응답결과를 반환
    if res.status_code not in error_code : return(res)
    
    # 일시적인 오류라면 예외 발생
    # 예외를 발생시켜야 retry가 작동한다.
    raise Exception('요청 오류 발생')

In [11]:
while True:
    try :
        time.sleep(1)
        
        # 요청
        res = fetch('http://httpbin.org/status/200,404,503,408,500,502,504')
        
        # 응답 코드에 따라 분기
        print(res.status_code)
        if 200<= res.status_code<300:
            print('성공')
            print('크롤링 코드')            
        else : 
            print('영구 오류..집에가자')
            break

        print('='*100)
            
    except Exception as e:
        print(e)
        print('일시적 오류 발생')
        print('오류가 발생한 주소를 파일에 저장해두세요')
        print('='*100)

응답코드 : 504
응답코드 : 504
응답코드 : 502
요청 오류 발생
일시적 오류 발생
오류가 발생한 주소를 파일에 저장해두세요
응답코드 : 502
응답코드 : 200
200
성공
크롤링 코드
응답코드 : 404
404
영구 오류..집에가자
